#Зчитати таблиці з папки bronze, провести обробку даних та записати в папку silver

In [0]:
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

# Функція для очищення тексту
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9,.\\"\']', '', str(text))

clean_text_udf = udf(clean_text, StringType())

In [0]:
import os

# Функція для створення директорії
def ensure_dir_exists(path):
    os.makedirs(path, exist_ok=True)

In [0]:
# Очищення текстових колонок, дедублікація та збереження
def process_table(input_path, output_path):
  
    df = spark.read.parquet(input_path)
    
    # Очищення текстових колонок
    for col_name, col_type in df.dtypes:
        if col_type == "string":
            df = df.withColumn(col_name, clean_text_udf(col(col_name)))
    
    # Дедублікація рядків
    df = df.dropDuplicates()

    # Перевірка наявності директорії
    ensure_dir_exists(output_path)
    
    # Збереження у форматі Parquet
    df.write.mode("overwrite").parquet(output_path)
    print(f"Таблиця збережена в: {output_path}")

    # Перепровірка наявності записаного датасету
    df = spark.read.parquet(output_path)
    df.show(truncate=False)

In [0]:
# Шляхи до папки bronze і silver
bronze_path = "/tmp/bronze/"
silver_path = "/tmp/silver/"

ensure_dir_exists(silver_path)

In [0]:
# Обробка таблиць
tables = ["athlete_bio", "athlete_event_results"]
for table in tables:
    input_path = f"{bronze_path}{table}"
    output_path = f"{silver_path}{table}"
    process_table(input_path, output_path)

Таблиця збережена в: /tmp/silver/athlete_bio
+----------+---------------+------+--------------+------+------+------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
dbutils.fs.ls("dbfs:/tmp/silver/")

[FileInfo(path='dbfs:/tmp/silver/athlete_bio/', name='athlete_bio/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/silver/athlete_event_results/', name='athlete_event_results/', size=0, modificationTime=0)]